In [41]:
import pandas as pd
from keras.utils import to_categorical

pd.set_option('display.max_rows', None)
import numpy  as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import keras
from tensorflow import keras
from keras.models import Model
from keras.metrics import Precision, Recall
from keras.layers import Dense, Input, Flatten, Dropout, Embedding

In [42]:
# Read in the csv data using pandas
def load_data():
    train = pd.read_csv('../data/01_train/train.csv',header=None)
    sampleTest   = pd.read_csv('../data/01_train/sampleTest.csv')
    sampleSubmission  = pd.read_csv('../data/01_train/sampleSubmission.csv')
    return train,sampleTest,sampleSubmission

In [43]:
#Feature Scaling
# load data
train, sampleTest, sampleSubmission = load_data()

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
train_scaled = sc.fit_transform(train)

In [44]:
x = []
y = []
n_future = 7 # next 4 days temperature forecast
n_past = 90 # Past 90 days
for j in range(0,len(train_scaled)):
    for i in range(0,len(train_scaled[j])-n_past-n_future+1):
        x.append(train_scaled[j,i : i + n_past ])
        y.append(train_scaled[j,i + n_past : i + n_past + n_future ])
x , y = np.array(x), np.array(y)
x = np.reshape(x, (x.shape[0] , x.shape[1], 1) )


x_train = []
y_train = []
n_future = 7 # next 4 days temperature forecast
n_past = 90 # Past 30 days
n_overlap=3
overlap=[]
for i in range(1,n_overlap+1):
    temp=int(n_past-(n_past/n_overlap)*(i))
    overlap.append(temp)

for j in range(0,len(train_scaled)):
    for i in range(0,(len(train_scaled[j])-n_past-n_future+1)):
        for o in overlap:
            von=i+o
            bis=i+o + n_past
            if(len(train_scaled[j,von : bis ])==90 and len(train_scaled[j,bis : bis + n_future ])==7):
                x_train.append(train_scaled[j,von : bis ])
                y_train.append(train_scaled[j,bis : bis + n_future ])

#Ah = np.vstack(x_train)
#Av = np.hstack(x_train)
#x_trainl=x_train
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )
x=x_train

In [45]:
from keras.models import Sequential
from keras.layers import LSTM,Dense ,Dropout
# Fitting RNN to training set using Keras Callbacks. Read Keras callbacks docs for more info.

In [46]:
from keras.layers import Bidirectional, RNN, SimpleRNN


def load_model(x_train):
    model = Sequential()
    model.add(Bidirectional(SimpleRNN(units=2, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
    model.add(Dropout(0.2))
    model.add(LSTM(units= 2 , return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units= 2 , return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(units= 2))
    model.add(Dropout(0.2))
    model.add(Dense(units = n_future,activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc','mean_squared_error'])
    return model


In [ ]:
# import model_selection module of scikit-learn
from sklearn import model_selection
# initiate the k-fold class from model_selection module
splits=2
#for i in range(splits):
kf = model_selection.KFold(n_splits=splits,shuffle=True)
for fold, (trn_, val_) in enumerate(kf.split(X=x)):
    #split dataset
    x_train=x[trn_]
    y_train=y[trn_]
    x_test=x[val_]
    y_test=y[val_]

    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)

    model=load_model(x_train)
    model.fit(x_train, y_train, epochs=200,batch_size=256,validation_data=(x_test, y_test))

(31986, 90, 1)
(31986, 7)
(31986, 90, 1)
(31986, 7)
Epoch 1/200
 22/125 [====>.........................] - ETA: 7s - loss: 0.4113 - acc: 0.1474 - mean_squared_error: 0.4113

KeyboardInterrupt: 

In [48]:
predicted_temperature = model.predict(x_test)
predicted_temperature = sc.inverse_transform(predicted_temperature)
#predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0]))


KeyboardInterrupt



In [ ]:
y_test

In [ ]:
predicted_temperature

In [ ]:
dif=y_test-predicted_temperature
dif

In [ ]:
from numpy import mean

mean(abs(dif))